In [2]:
import sys
sys.executable

'/home/ubuntu/anaconda3/envs/shun/bin/python'

In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
from fastai import *
from fastai.vision import *